<a href="https://colab.research.google.com/github/JeevithaH/dataset/blob/main/Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
Found existing installation: accelerate 0.20.3
Uninstalling accelerate-0.20.3:
  Successfully uninstalled accelerate-0.20.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)


In [ ]:
from transformers import pipeline,set_seed
from datasets import load_dataset,load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset,load_metric
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch
nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
#download & unzip data
#using SAMsum data
!wget https://github.com/JeevithaH/dataset/archive/refs/heads/main.zip



--2023-06-23 16:18:54--  https://github.com/JeevithaH/dataset/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/JeevithaH/dataset/zip/refs/heads/main [following]
--2023-06-23 16:18:54--  https://codeload.github.com/JeevithaH/dataset/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip.1’

main.zip.1              [  <=>               ]   7.53M  31.2MB/s    in 0.2s    

2023-06-23 16:18:55 (31.2 MB/s) - ‘main.zip.1’ saved [7897133]



In [ ]:
!unzip /content/main.zip


Archive:  /content/main.zip
deed9b90eef0544a64931c690795529668e9eff2
replace dataset-main/summarizer-data.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip /content/dataset-main/summarizer-data.zip

Archive:  /content/dataset-main/summarizer-data.zip
replace samsum-test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
dataset_samsum = load_from_disk("samsum_dataset")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
split_lengths=[len(dataset_samsum[split])for split in dataset_samsum]

print(f"split lengths:{split_lengths}")
print(f"Features:{dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

split lengths:[14732, 819, 818]
Features:['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [ ]:
def convert_example_to_features(example_batch):
  input_encodings= tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)
  with tokenizer.as_target_tokenizer():
    target_encodings=tokenizer(example_batch['summary'],max_length=128, truncation=True)

  return{
      'input_ids':input_encodings['input_ids'],
      'attention_mask':input_encodings['attention_mask'],
      'labels':target_encodings["input_ids"]
  }

In [ ]:
dataset_samsum_pt = dataset_samsum.map(convert_example_to_features ,batched=True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [ ]:
#training
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args =TrainingArguments(output_dir='pegasus-samsum',num_train_epochs=1,warmup_steps=500,
    per_device_train_batch_size=1,per_device_eval_batch_size=1,
    weight_decay=0.01,logging_steps=10,
    evaluation_strategy='steps',eval_steps=500,save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer =Trainer(model=model_pegasus,args=trainer_args,tokenizer=tokenizer,
                data_collator=seq2seq_data_collator,train_dataset=dataset_samsum_pt["train"],
                eval_dataset=dataset_samsum_pt["validation"])

In [19]:
trainer.train()

Step,Training Loss,Validation Loss
500,1.514000,1.428929


TrainOutput(global_step=920, training_loss=1.5529723271079685, metrics={'train_runtime': 3012.0973, 'train_samples_per_second': 4.891, 'train_steps_per_second': 0.305, 'total_flos': 5578032042835968.0, 'train_loss': 1.5529723271079685, 'epoch': 1.0})

In [20]:
# Evaluation
def generate_batch_sized_chunks(list_of_elements,batch_size):
  """split the dataset into smaller batches that we can process simultaneously
  view successive batch-sized chunks from list_of_elements,"""
  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i:i+batch_size]

def calculate_metric_on_test_ds(dataset,metric,model,tokenizer,batch_size=16,device=device,
                                column_text="article",column_summary="highlights"):
  article_batches=list(generate_batch_sized_chunks(dataset[column_text],batch_size))
  target_batches=list(generate_batch_sized_chunks(dataset[column_summary],batch_size))

  for article_batch,target_batch in tqdm(
      zip(article_batches,target_batches),total=len(article_batches)):

      inputs = tokenizer(article_batch,max_length=1024,truncation=True,
                        padding="max_length",return_tensors="pt")

      summaries=model.generate(input_ids=inputs["input_ids"].to(device),
                               attention_mask=inputs["attention_mask"].to(device),
                               length_penalty=0.8,num_beams=8,max_length=128)
      '''parameter for length penalty ensure that the model does not generate sequence:'''
      #Finally.we decode the generated text,
      #replace the token,and add the decoded texts with the references to the matric,
      decoded_summaries=[tokenizer.decode(s,skip_special_tokens=True,
                                          clean_up_tokenization_spaces=True)
              for s in summaries]

      decoded_summarizes=[d.replace(""," ") for d in decoded_summaries]

      metric.add_batch(predictions=decoded_summaries,references = target_batch)

      #finally compute and return thr ROUGE scores.
      score=metric.compute()
      return score



In [21]:
rouge_names=["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric=load_metric("rouge")

<ipython-input-21-d244ab2fd21d>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric=load_metric("rouge")


In [22]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'],rouge_metric,trainer.model,tokenizer,batch_size=2,
    column_text='dialogue',column_summary='summary'
)
rouge_dict=dict((rn, score[rn].mid.fmeasure)for rn in rouge_names)
pd.DataFrame(rouge_dict,index =[f"pegasus"])

  0%|          | 0/410 [00:02<?, ?it/s]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.406034,0.163066,0.332495,0.332495


In [23]:
# save model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [24]:
#save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [25]:
#load
tokenizer=AutoTokenizer.from_pretrained("/content/tokenizer")

In [26]:
#prediction
gen_kwargs={"length_penalty":0.8,"num_beams":8,"max_length":128}

sample_text=dataset_samsum["test"][0]["dialogue"]
reference=dataset_samsum["test"][0]["summary"]
pipe=pipeline("summarization",model="pegasus-samsum-model",tokenizer = tokenizer)
##
print("Dialogue:")
print(sample_text)
print("\nReference Summary")
print(reference)
print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda can't find Betty's number. Larry called Betty the last time they were at the park together. Hannah wants Amanda to text Larry instead.


In [27]:
#prediction
gen_kwargs={"length_penalty":0.8,"num_beams":8,"max_length":40}

sample_text=dataset_samsum["test"][0]["dialogue"]
reference=dataset_samsum["test"][0]["summary"]
pipe=pipeline("summarization",model="pegasus-samsum-model",tokenizer = tokenizer)
##
print("Dialogue:")
print(sample_text)
print("\nReference Summary")
print(reference)
print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda can't find Betty's number. Larry called Betty the last time they were at the park together. Hannah wants Amanda to text Larry instead.
